In [157]:
#!g1.1
%pip install pandas numpy torch scikit-learn nltk torchmetrics transformers datasets seaborn pytorch-lightning

Defaulting to user installation because normal site-packages is not writeable
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
moto 1.3.14 requires idna<2.9,>=2.5, but you have idna 3.4 which is incompatible.
kaggle 1.5.8 requires urllib3<1.25,>=1.21.1, but you have urllib3 1.26.15 which is incompatible.
cloud-ml 0.0.1 requires requests<=2.25.1,>=2.22.0, but you have requests 2.28.2 which is incompatible.
cloud-ml 0.0.1 requires tqdm<=4.54.1,>=4.45.0, but you have tqdm 4.65.0 which is incompatible.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [158]:
#!g1.1
dataset_path = "/home/jupyter/mnt/s3/databucket"

In [159]:
#!g1.1
%ls $dataset_path

Unigram Train-Copy1.png*
Unigram Train.png*
Untitled Folder/
bank-categories.csv*
bank-sentiment.csv*
banki.csv*
banks-ethic-DB - NEW.docx*
checkpoints/
foo.pdf*
foo.png*
lightning_logs/
pie.png*
sentiment.csv*
source/
to.png*
train.csv*


# Imports

In [160]:
#!g1.1
import pandas as pd
import numpy as np
import re

import torch
from torchmetrics import AUROC
from torchmetrics import ROC
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from nltk.stem.snowball import RussianStemmer
from nltk.tokenize import TweetTokenizer

In [161]:
#!g1.1
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.preprocessing
from tqdm.notebook import tqdm

from functools import lru_cache
import os
import time
import glob
import copy
import random
import shutil
import requests
import subprocess
from urllib.request import urlretrieve
from typing import Any, Callable, Dict, Iterable, List, Tuple
from collections import defaultdict, OrderedDict

from sklearn.model_selection import train_test_split

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
import pytorch_lightning as pl
import transformers
import datasets

# Библиотеки для визуализации
import ipywidgets as widgets
from IPython.display import clear_output

import plotly 
import plotly.express as px
import seaborn as sns

from sklearn.utils.random import sample_without_replacement
from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.filterwarnings("ignore")

from joblib import load, dump

# seed_everything(42)

# Constants

In [162]:
#!g1.1
data = os.path.join(dataset_path, 'banki.csv')
data

'/home/jupyter/mnt/s3/databucket/banki.csv'

# Load and Prepare Data

In [163]:
#!g1.1
tweets_col_number = 3

dataset = pd.read_csv(os.path.join(dataset_path, 'train.csv'), index_col=0)
#positive_tweets = pd.read_csv(
#    'positive.csv', header=None, delimiter=';')[[tweets_col_number]]
dataset_text=dataset.drop(['1category','2category'],axis=1)
data_positive=dataset_text[dataset_text['sentiment']=='+']
data_negative=dataset_text[dataset_text['sentiment']=='−']
data_neutral=dataset_text[dataset_text['sentiment']=='?']

In [164]:
#!g1.1
print(len(data_positive),len(data_negative),len(data_neutral))

6262 10192 2907


In [165]:
#!g1.1
double_2cat = dataset[~dataset["2category"].isna()]
double_2cat["1category"] = double_2cat["2category"]
double_2cat["2category"] = np.nan
double_2cat

,sentence,1category,2category,sentiment
13007,"Начну с того, что я пользовался и пользуюсь ус...",Quality,NaN,+
18828,Точка идеально походит для таких «чайников» ка...,Communication,NaN,+
19214,"Открывали счет 2 недели... Открыли, пока готов...",Price,NaN,−
19220,Итого что имеем обещанная ставка выросла более...,Price,NaN,−
18944,"Резюме: не ходите в Росбанк, он очень непорядо...",Communication,NaN,−
...,...,...,...,...
12378,Почему работают неквалифицированные специалист...,Quality,NaN,−
18821,Это реально круто.2) Очень грамотные менеджеры...,Communication,NaN,+
12593,"Ответа Банка я так и не получила, и, хуже того...",Safety,NaN,−
4600,* Удобство: 10 из 10* Работа сотрудников: 10 и...,Quality,NaN,+


In [166]:
#!g1.1
dataset = pd.concat((dataset, double_2cat)).drop(columns="2category").drop_duplicates()
dataset

,sentence,1category,sentiment
4754,При этом всегда получал качественные услуги.,Communication,+
4417,"Не вижу, за что хотя бы 2 поставить, сервис на 1!",?,−
3629,"Вот так ""Мой любимый"" банк МКБ меня обманул.",?,−
11640,Отвратительное отношение к клиентам.,Communication,−
5571,"Всегда в любое время дня и ночи помогут, ответ...",Communication,+
...,...,...,...
19199,"В очереди, кстати, узнал, что многие также зак...",Price,−
12512,"Мало того, что услуги навязали, так еще и вопр...",Quality,−
19056,Почему-то с этим банком постоянно возникают пр...,Communication,−
12940,Что по статистике такие суммы сейчас вообще не...,Communication,−


# Bert-like модель

In [167]:
#!g1.1
model_name = "blanchefort/rubert-base-cased-sentiment-rusentiment"

In [168]:
#!g1.1
from transformers import AutoModel, AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True)

Downloading:   0%|          | 0.00/495 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/952 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679M [00:00<?, ?B/s]

In [169]:
#!g1.1
class MultiTaskClassifier(nn.Module):
    def __init__(self, in_features=768, n_classes=[3, 4], bias=True):
        super(MultiTaskClassifier, self).__init__()
        self.in_features = in_features
        self.n_classes = n_classes
        
        self.classifiers = nn.ModuleList([nn.Linear(in_features, n_class, bias=bias) for n_class in n_classes])
        
    def forward(self, x):
        return (classifier(x) for classifier in self.classifiers)

In [170]:
#!g1.1
model.classifier = MultiTaskClassifier()

In [171]:
#!g1.1
CLS_TOKEN = [tokenizer.cls_token_id]
SEP_TOKEN = [tokenizer.sep_token_id]
samples = dataset.sample(2).sentence.values

tokenized = tokenizer(
    samples.tolist() + ["Привет, мир!"],
    max_length=512,
    padding=True,
    truncation=True,
    return_tensors='pt',
)

tokenized, CLS_TOKEN, SEP_TOKEN, tokenizer.batch_decode(tokenized.input_ids)

({'input_ids': tensor([[  101,  7289,  5979,  6193,   128,  1129,  1250,   322,   802,  3604,
           4212,   340, 75879,   132,   102,     0,     0,     0,     0],
         [  101, 29179,  3191,   130,  1248,  1094,  8238,   128,   301,   336,
          23533,  4299,  1447,  8019,   130,  1094,  8238,   132,   102],
         [  101,  4152,   128,  2498,   106,   102,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])},
 [101],
 [102],
 ['[CLS] Бардак полный, после этого и не хочется идти в сбе

In [172]:
#!g1.1
dataset["1category"].unique()

array(['Communication', '?', 'Quality', 'Price', 'Safety'], dtype=object)

In [173]:
#!g1.1
dataset['target1'] = dataset.sentiment.apply(lambda x: {"−":0, "?":1, "+":2}[x])
dataset['target2'] = dataset["1category"].apply(lambda x: int({"Communication":0, "Quality":1, "Price":2, "Safety":2, '?': -1}[x]))
dataset

,sentence,1category,sentiment,target1,target2
4754,При этом всегда получал качественные услуги.,Communication,+,2,0
4417,"Не вижу, за что хотя бы 2 поставить, сервис на 1!",?,−,0,-1
3629,"Вот так ""Мой любимый"" банк МКБ меня обманул.",?,−,0,-1
11640,Отвратительное отношение к клиентам.,Communication,−,0,0
5571,"Всегда в любое время дня и ночи помогут, ответ...",Communication,+,2,0
...,...,...,...,...,...
19199,"В очереди, кстати, узнал, что многие также зак...",Price,−,0,2
12512,"Мало того, что услуги навязали, так еще и вопр...",Quality,−,0,1
19056,Почему-то с этим банком постоянно возникают пр...,Communication,−,0,0
12940,Что по статистике такие суммы сейчас вообще не...,Communication,−,0,0


In [174]:
#!g1.1
class TextDataset(Dataset):
    def __init__(self, data: pd.DataFrame, tokenizer, pre_tokenizing=False):
        self.data = data

        self.texts = data.sentence.tolist()
        self.labels1 = data.target1.tolist()
        self.labels2 = data.target2.tolist()
        self.tokenizer = tokenizer
        self.pre_tokenizing = pre_tokenizing
        
        if pre_tokenizing:
            self.tokenized_texts = [self.tokenize(text) for text in tqdm(self.texts)]
    
    def __len__(self):
        return len(self.data)
    
    def tokenize(self, text):
        return tokenizer(
            text,
            max_length=512,
            padding=True,
            truncation=True,
            return_tensors='pt',
        )

    def __getitem__(self, item: int):
        label1 = self.labels1[item]
        label2 = self.labels2[item]
        if self.pre_tokenizing:
            tokenized = self.tokenized_texts[item]
        else:
            tokenized = self.tokenize(self.texts[item])
        return tokenized.input_ids.squeeze(dim=0), tokenized.attention_mask.squeeze(dim=0), torch.LongTensor([label1]), torch.LongTensor([label2])   

def my_collate_fn(batch):
    tokens, attention_masks, labels1, labels2 = list(zip(*batch)) #, document_ids, sentences_ids = list(zip(*batch))
    
    tokens = pad_sequence(tokens, batch_first=True, padding_value=0)
    attention_masks = pad_sequence(attention_masks, batch_first=True, padding_value=0)
    labels1 = torch.cat(labels1)
    labels2 = torch.cat(labels2)
#     labels = pad_sequence(labels, batch_first=True, padding_value=-100)

    return tokens, attention_masks, labels1, labels2 #, document_ids, sentences_ids  

In [175]:
#!g1.1
bs = 8

train_loader = DataLoader(
    TextDataset(
        dataset[:128],
        tokenizer,
        pre_tokenizing=True,
    ),
    shuffle=True,
    batch_size=bs,
    collate_fn=my_collate_fn,
)

  0%|          | 0/128 [00:00<?, ?it/s]

In [176]:
#!g1.1
tokens, masks, labels1, labels2 = next(iter(train_loader))
tokens.shape, masks.shape, labels1.shape, labels2.shape

(torch.Size([8, 44]), torch.Size([8, 44]), torch.Size([8]), torch.Size([8]))

In [177]:
#!g1.1
class BERTSentimentClassifier(nn.Module):
    def __init__(self, model):
        super(BERTSentimentClassifier, self).__init__()
        self.model = model
        
    def forward(self, input_ids, attention_mask=None):
        return list(self.model(input_ids, attention_mask=attention_mask).logits)

model = BERTSentimentClassifier(model)
model(tokens, masks)

[tensor([[-0.0186, -0.2031,  0.0031],
         [ 0.1186,  0.4424,  0.5632],
         [ 0.2765,  0.5597,  0.3002],
         [-0.0345, -0.1821, -0.0096],
         [-0.1083, -0.3894, -0.0736],
         [-0.0284, -0.3166, -0.0480],
         [-0.0178,  0.0606,  0.3956],
         [ 0.0177, -0.1018,  0.0492]], grad_fn=<AddmmBackward>),
 tensor([[-0.1552,  0.2531,  0.6723, -0.1176],
         [-0.1279,  0.0773,  0.0065,  0.3774],
         [ 0.2090, -0.3031, -0.3114,  0.5013],
         [-0.0987,  0.2325,  0.6246, -0.0036],
         [-0.1642,  0.3122,  0.6606, -0.2224],
         [-0.0750,  0.2882,  0.6345, -0.1894],
         [-0.3039,  0.2397,  0.5655,  0.1618],
         [-0.1057,  0.1758,  0.6512,  0.0637]], grad_fn=<AddmmBackward>)]

In [178]:
#!g1.1
sent_train, sent_valid = train_test_split(dataset.sentence.unique(), train_size=0.8)
sent_valid, sent_test = train_test_split(sent_valid, train_size=0.5)

In [179]:
#!g1.1
df_train = dataset[dataset.sentence.isin(sent_train)]
df_valid = dataset[dataset.sentence.isin(sent_valid)]
df_test = dataset[dataset.sentence.isin(sent_test)]

df_train.shape, df_valid.shape, df_test.shape

((10307, 5), (1293, 5), (1271, 5))

In [200]:
#!g1.1
import gc
from sklearn.metrics import f1_score

class LightningBERT(pl.LightningModule):
    def __init__(self, model, optim_params=dict(), batch_size=8,
                 save_path=None, save_name=None, save_period=1000, check_all=False,
                 loss_coef=1.0, pre_tokenizing=False, log_period=10):
        super().__init__()

        self.model = model
        self.optim_params = optim_params
        self.batch_size = batch_size
        self.save_path = save_path
        self.save_period = save_period
        self.save_name = save_name
        self.check_all = check_all
        self.loss_coef = loss_coef
        self.pre_tokenizing = pre_tokenizing
        self.log_period = log_period

        # make variables for storing true and pred labels from each batch
        self.train_loss = 0
        self.train_preds1 = []
        self.train_preds2 = []
        self.train_labels1 = []
        self.train_labels2 = []
        self.train_metric1 = []
        self.train_metric2 = []
        
        self.valid_loss = 0
        self.valid_preds1 = []
        self.valid_preds2 = []
        self.valid_labels1 = []
        self.valid_labels2 = []
        self.valid_metric1 = []
        self.valid_metric2 = []
        
        self.test_loss = 0
        self.test_preds1 = []
        self.test_preds2 = []
        self.test_labels1 = []
        self.test_labels2 = []
        self.test_metric1 = []
        self.test_metric2 = []

        self.criterion = nn.CrossEntropyLoss(ignore_index =-1)
        self.auroc1 = AUROC(num_classes=3)
        self.auroc2 = AUROC(num_classes=4)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(filter(lambda param: param.requires_grad, self.model.parameters()), **self.optim_params)
        return optimizer

    def compute_loss(self, logits, labels):
        return self.criterion(logits, labels)
    
    def compute_metrics1(self, preds, labels):
        return self.auroc1(preds, labels)
    
    def compute_metrics2(self, preds, labels):
        cond = labels != -1
        if torch.sum(cond) == 0:
            return 0
        return self.auroc2(preds[cond], labels[cond])
    
    def training_step(self, batch, batch_idx):
        tokens, masks, labels1, labels2 = batch
        
        logits1, logits2 = self.model(tokens, masks)
        loss = self.compute_loss(logits1, labels1) + self.loss_coef * self.compute_loss(logits2, labels2)

        self.train_loss += loss.detach().cpu().numpy()
        self.train_preds1.append(F.softmax(logits1, dim=-1).detach().cpu())
        self.train_preds2.append(F.softmax(logits2, dim=-1).detach().cpu())
        self.train_labels1.append(labels1.detach().cpu())
        self.train_labels2.append(labels2.detach().cpu())
        
        self.train_metric1.append(self.compute_metrics1(self.train_preds1[-1], self.train_labels1[-1]))
        self.train_metric2.append(self.compute_metrics2(self.train_preds2[-1], self.train_labels2[-1]))
        
        if (batch_idx + 1) % self.log_period == 0:
            self.log("iter_train_loss:", loss, prog_bar=True)
            self.log("iter_train_metric1:", np.mean(self.train_metric1), prog_bar=True)
            self.log("iter_train_metric2:", np.mean(self.train_metric2), prog_bar=True)

        return loss

    def training_epoch_end(self, _):
        self.train_loss /= self.train_len
        metric1 = np.mean(self.train_metric1)
        metric2 = np.mean(self.train_metric2)
        
        self.log("Train Loss:", self.train_loss, prog_bar=True)
        self.log("Train Metric 1:", metric1, prog_bar=True)
        self.log("Train Metric 2:", metric2, prog_bar=True)
        print("Train Loss: ", self.train_loss)
        print("Train Metric 1:", metric1)
        print("Train Metric 2:", metric2)

        self.train_loss = 0
        self.train_preds1 = []
        self.train_preds2 = []
        self.train_labels1 = []
        self.train_labels2 = []
        self.train_metric1 = []
        self.train_metric2 = []
        
        torch.cuda.empty_cache()
        gc.collect()

    def validation_step(self, batch, batch_idx):
        tokens, masks, labels1, labels2 = batch
        
        with torch.no_grad():
            logits1, logits2 = self.model(tokens, masks)
            loss = self.compute_loss(logits1, labels1) + self.loss_coef * self.compute_loss(logits2, labels2)

        self.valid_loss += loss.detach().cpu().numpy()
        self.valid_preds1.append(F.softmax(logits1, dim=-1).detach().cpu())
        self.valid_preds2.append(F.softmax(logits2, dim=-1).detach().cpu())
        self.valid_labels1.append(labels1.detach().cpu())
        self.valid_labels2.append(labels2.detach().cpu())
        
        self.valid_metric1.append(self.compute_metrics1(self.valid_preds1[-1], self.valid_labels1[-1]))
        self.valid_metric2.append(self.compute_metrics2(self.valid_preds2[-1], self.valid_labels2[-1]))
        
        if (batch_idx + 1) % self.log_period == 0:
            self.log("iter_valid_loss:", loss, prog_bar=True)
            self.log("iter_valid_metric1:", np.mean(self.valid_metric1), prog_bar=True)
            self.log("iter_valid_metric2:", np.mean(self.valid_metric2), prog_bar=True)

        return loss

    def validation_epoch_end(self, _):
        self.valid_loss /= self.valid_len
        metric1 = np.mean(self.valid_metric1)
        metric2 = np.mean(self.valid_metric2)
        
        self.log("Valid Loss:", self.valid_loss, prog_bar=True)
        self.log("Valid Metric 1:", metric1, prog_bar=True)
        self.log("Valid Metric 2:", metric2, prog_bar=True)
        print("Valid Loss: ", self.valid_loss)
        print("Valid Metric 1:", metric1)
        print("Valid Metric 2:", metric2)

        self.valid_loss = 0
        self.valid_preds1 = []
        self.valid_preds2 = []
        self.valid_labels1 = []
        self.valid_labels2 = []
        self.valid_metric1 = []
        self.valid_metric2 = []
        
        torch.cuda.empty_cache()
        gc.collect()

    def test_step(self, batch, batch_idx):
        tokens, masks, labels1, labels2 = batch
        
        with torch.no_grad():
            logits1, logits2 = self.model(tokens, masks)
            loss = self.compute_loss(logits1, labels1) + self.loss_coef * self.compute_loss(logits2, labels2)

        self.test_loss += loss.detach().cpu().numpy()
        self.test_preds1.append(F.softmax(logits1, dim=-1).detach().cpu())
        self.test_preds2.append(F.softmax(logits2, dim=-1).detach().cpu())
        self.test_labels1.append(labels1.detach().cpu())
        self.test_labels2.append(labels2.detach().cpu())
        
        self.test_metric1.append(self.compute_metrics1(self.test_preds1[-1], self.test_labels1[-1]))
        self.test_metric2.append(self.compute_metrics2(self.test_preds2[-1], self.test_labels2[-1]))
        
        if (batch_idx + 1) % self.log_period == 0:
            self.log("iter_test_loss:", loss, prog_bar=True)
            self.log("iter_test_metric1:", np.mean(self.test_metric1), prog_bar=True)
            self.log("iter_test_metric2:", np.mean(self.test_metric2), prog_bar=True)
        
        return loss

    def test_epoch_end(self, _):
        self.test_loss /= self.test_len
        metric1 = np.mean(self.test_metric1)
        metric2 = np.mean(self.test_metric2)
        
        self.log("Test Loss:", self.test_loss, prog_bar=True)
        self.log("Test Metric 1:", metric1, prog_bar=True)
        self.log("Test Metric 2:", metric2, prog_bar=True)
        print("Test Loss: ", self.test_loss)
        print("Test Metric 1:", metric1)
        print("Test Metric 2:", metric2)

        self.test_loss = 0
        self.test_preds1 = []
        self.test_preds2 = []
        self.test_labels1 = []
        self.test_labels2 = []
        self.test_metric1 = []
        self.test_metric2 = []
        
        torch.cuda.empty_cache()
        gc.collect()

    def train_dataloader(self):
        train_loader = DataLoader(
            TextDataset(
                df_train[:128] if self.check_all else df_train,
                tokenizer=tokenizer, pre_tokenizing=self.pre_tokenizing
            ),
            shuffle=True,
            batch_size=self.batch_size,
            collate_fn=my_collate_fn,
            num_workers=8,
        )
        
        self.train_len = len(train_loader)
        return train_loader

    def val_dataloader(self):
        valid_loader = DataLoader(
            TextDataset(
                df_valid[:128] if self.check_all else df_valid,
                tokenizer=tokenizer, pre_tokenizing=self.pre_tokenizing
            ),
            shuffle=False,
            batch_size=self.batch_size,
            collate_fn=my_collate_fn,
            num_workers=8,
        )
        
        self.valid_len = len(valid_loader)
        return valid_loader

    def test_dataloader(self):
        test_loader = DataLoader(
            TextDataset(
                df_test[:128] if self.check_all else df_test,
                tokenizer=tokenizer, pre_tokenizing=self.pre_tokenizing
            ),
            shuffle=False,
            batch_size=self.batch_size,
            collate_fn=my_collate_fn,
            num_workers=8,
        )
        
        self.test_len = len(test_loader)
        return test_loader

In [201]:
#!g1.1
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [202]:
#!g1.1
gc.collect()
torch.cuda.empty_cache()

In [189]:
#!g1.1
from pytorch_lightning import Trainer

lightning_model = LightningBERT(
    model=model,
    optim_params={"lr":1e-4},
    loss_coef=5.0,
    batch_size=8,
#     save_path="./checkpoints1",
#     save_name="model",
#     save_period=6000,
#     check_all=True,
    pre_tokenizing=True,
)

trainer = Trainer(
    gpus=-1,
    accelerator='gpu',
    max_epochs=3,
    gradient_clip_val=1.0,
    val_check_interval=0.5,
    accumulate_grad_batches=8,
)

trainer.fit(lightning_model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                    | Params
------------------------------------------------------
0 | model     | BERTSentimentClassifier | 177 M 
1 | criterion | CrossEntropyLoss        | 0     
2 | auroc1    | AUROC                   | 0     
3 | auroc2    | AUROC                   | 0     
------------------------------------------------------
177 M     Trainable params
0         Non-trainable params
177 M     Total params
711.435   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

  0%|          | 0/1293 [00:00<?, ?it/s]

Valid Loss:  0.04385051168041465
Valid Metric 1: 0.67777777
Valid Metric 2: 0.39583334


  0%|          | 0/10307 [00:00<?, ?it/s]

...

Valid Loss:  4.175564510218891
Valid Metric 1: 0.8387909
Valid Metric 2: 0.3717761
Train Loss:  3.9029870087828535
Train Metric 1: 0.8303795
Train Metric 2: 0.41517207


Validating: 0it [00:00, ?it/s]

Valid Loss:  4.3325174907107415
Valid Metric 1: 0.8305885
Valid Metric 2: 0.3746295


Validating: 0it [00:00, ?it/s]

Valid Loss:  4.369262061369272
Valid Metric 1: 0.8335917
Valid Metric 2: 0.35921302
Train Loss:  3.6022724988758332
Train Metric 1: 0.8403559
Train Metric 2: 0.43378302


In [203]:
#!g1.1
for parameter in model.parameters():
    parameter.requires_grad = True

In [204]:
#!g1.1
lightning_model = LightningBERT(
    model=model,
    optim_params={"lr":3e-6},
    loss_coef=5.0,
    batch_size=8,
#     save_path="./checkpoints1",
#     save_name="model",
#     save_period=6000,
#     check_all=True,
    pre_tokenizing=True,
)

trainer = Trainer(
    gpus=-1,
    accelerator='gpu',
    max_epochs=5,
    val_check_interval=1.0,
    accumulate_grad_batches=4,
)

trainer.fit(lightning_model)
trainer.test(lightning_model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                    | Params
------------------------------------------------------
0 | model     | BERTSentimentClassifier | 177 M 
1 | criterion | CrossEntropyLoss        | 0     
2 | auroc1    | AUROC                   | 0     
3 | auroc2    | AUROC                   | 0     
------------------------------------------------------
177 M     Trainable params
0         Non-trainable params
177 M     Total params
711.435   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

  0%|          | 0/1293 [00:00<?, ?it/s]

Valid Loss:  0.04724957619184329
Valid Metric 1: 0.7222222
Valid Metric 2: 0.39583334


  0%|          | 0/10307 [00:00<?, ?it/s]

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


  0%|          | 0/1271 [00:00<?, ?it/s]

Testing: 0it [00:00, ?it/s]

Test Loss:  5.017196777481703
Test Metric 1: 0.81699914
Test Metric 2: 0.40144348
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'Test Loss:': 5.0171966552734375,
 'Test Metric 1:': 0.8169991374015808,
 'Test Metric 2:': 0.4014434814453125,
 'iter_test_loss:': 5.868220806121826,
 'iter_test_metric1:': 0.8456400632858276,
 'iter_test_metric2:': 0.4057683050632477}
--------------------------------------------------------------------------------


[{'iter_test_loss:': 5.868220806121826,
  'iter_test_metric1:': 0.8456400632858276,
  'iter_test_metric2:': 0.4057683050632477,
  'Test Loss:': 5.0171966552734375,
  'Test Metric 1:': 0.8169991374015808,
  'Test Metric 2:': 0.4014434814453125}]

In [ ]:
#!g1.1
